In [1]:
import numpy as np
import networkx as nx

from graphqec import RepetitionCode, RotatedSurfaceCode, CssCode, BivariateBicycleCode,commutation_test

In [2]:
bbc = BivariateBicycleCode(
    Lx = 6,
    Ly = 6,
    a1 = 3,
    a2 = 1,
    a3 = 2,
    b1 = 3,
    b2 = 1,
    b3 = 2,
)

In [3]:
A = bbc.A1+bbc.A2+bbc.A3
B = bbc.B1+bbc.B2+bbc.B3

In [4]:
Hx = np.hstack((bbc.A,bbc.B))
Hz = np.hstack((bbc.B.T, bbc.A.T))

In [5]:
cssbbc = CssCode(Hx,Hz)

In [6]:
dict(cssbbc._graph.nodes(data=True))

{0: {'type': 'data', 'label': None, 'label_id': 0},
 1: {'type': 'data', 'label': None, 'label_id': 1},
 2: {'type': 'data', 'label': None, 'label_id': 2},
 3: {'type': 'data', 'label': None, 'label_id': 3},
 4: {'type': 'data', 'label': None, 'label_id': 4},
 5: {'type': 'data', 'label': None, 'label_id': 5},
 6: {'type': 'data', 'label': None, 'label_id': 6},
 7: {'type': 'data', 'label': None, 'label_id': 7},
 8: {'type': 'data', 'label': None, 'label_id': 8},
 9: {'type': 'data', 'label': None, 'label_id': 9},
 10: {'type': 'data', 'label': None, 'label_id': 10},
 11: {'type': 'data', 'label': None, 'label_id': 11},
 12: {'type': 'data', 'label': None, 'label_id': 12},
 13: {'type': 'data', 'label': None, 'label_id': 13},
 14: {'type': 'data', 'label': None, 'label_id': 14},
 15: {'type': 'data', 'label': None, 'label_id': 15},
 16: {'type': 'data', 'label': None, 'label_id': 16},
 17: {'type': 'data', 'label': None, 'label_id': 17},
 18: {'type': 'data', 'label': None, 'label_id':

In [7]:
cssbbc.nqubits

72

In [8]:
len(cssbbc.Hx)

36

[0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [3]:
rep = RepetitionCode(
    distance = 5,
    depolarize1_rate = 0.01,
    depolarize2_rate = 0.01
)

In [26]:
rot = RotatedSurfaceCode(
    distance = 5,
    depolarize1_rate = 0.01,
    depolarize2_rate = 0.01
)

0
1
2
3
4
5
6
7
8


In [17]:
rep.get_check_label

<bound method BaseCode.get_check_label of <graphqec.codes.repetition_code.RepetitionCode object at 0x000001FED1909D50>>

In [23]:
for node in graph.nodes():
    print(nx.get_node_attributes(graph,node))

{}
{}
{}
{}
{}
{}
{}
{}
{}
